Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [2]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2022-11-18 09:48:50,092: INFO: running /Users/masaboe/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py -f /Users/masaboe/Library/Jupyter/runtime/kernel-50d92442-b552-4d43-9f91-9d6d683eea61.json
2022-11-18 09:49:40,958: INFO: Saved 10000 articles
2022-11-18 09:50:15,960: INFO: Saved 20000 articles
2022-11-18 09:50:44,176: INFO: Saved 30000 articles
2022-11-18 09:51:09,549: INFO: Saved 40000 articles
2022-11-18 09:51:37,485: INFO: Saved 50000 articles
2022-11-18 09:52:01,574: INFO: Saved 60000 articles
2022-11-18 09:52:26,415: INFO: Saved 70000 articles
2022-11-18 09:52:53,762: INFO: Saved 80000 articles
2022-11-18 09:53:24,188: INFO: Saved 90000 articles
2022-11-18 09:53:50,359: INFO: Saved 100000 articles
2022-11-18 09:54:16,250: INFO: Saved 110000 articles
2022-11-18 09:54:38,439: INFO: Saved 120000 articles
2022-11-18 09:55:08,628: INFO: Saved 130000 articles
2022-11-18 09:55:33,918: INFO: Saved 140000 articles
2022-11-18 09:55:51,068: INFO: Saved 150000 articles
2022-11-

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [4]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2022-11-18 10:13:41,559: INFO: running /Users/masaboe/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py -f /Users/masaboe/Library/Jupyter/runtime/kernel-50d92442-b552-4d43-9f91-9d6d683eea61.json
2022-11-18 10:13:41,569: INFO: collecting all words and their counts
2022-11-18 10:13:41,572: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-18 10:13:44,450: INFO: PROGRESS: at sentence #10000, processed 10152030 words, keeping 346360 word types
2022-11-18 10:13:46,175: INFO: PROGRESS: at sentence #20000, processed 16285071 words, keeping 478960 word types
2022-11-18 10:13:47,510: INFO: PROGRESS: at sentence #30000, processed 21007490 words, keeping 579996 word types
2022-11-18 10:13:48,693: INFO: PROGRESS: at sentence #40000, processed 25264833 words, keeping 662876 word types
2022-11-18 10:13:49,986: INFO: PROGRESS: at sentence #50000, processed 29742480 words, keeping 760871 word types
2022-11-18 10:13:51,149: INFO: PROGRESS: at sentence #60000, pr

2022-11-18 10:14:36,673: INFO: EPOCH 0 - PROGRESS: at 0.21% examples, 424275 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:37,688: INFO: EPOCH 0 - PROGRESS: at 0.29% examples, 486619 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:38,689: INFO: EPOCH 0 - PROGRESS: at 0.41% examples, 515555 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:39,699: INFO: EPOCH 0 - PROGRESS: at 0.55% examples, 522829 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:40,702: INFO: EPOCH 0 - PROGRESS: at 0.71% examples, 520523 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:41,704: INFO: EPOCH 0 - PROGRESS: at 0.82% examples, 530498 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:42,707: INFO: EPOCH 0 - PROGRESS: at 0.95% examples, 545123 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:43,720: INFO: EPOCH 0 - PROGRESS: at 1.11% examples, 557628 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:14:44,721: INFO: EPOCH 0 - PROGRESS: at 1.31% examples, 567165 words/s, in_qsize 7, out_qsize 0
2022-11-18

2022-11-18 10:15:52,509: INFO: EPOCH 0 - PROGRESS: at 23.21% examples, 580246 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:15:53,519: INFO: EPOCH 0 - PROGRESS: at 23.60% examples, 579909 words/s, in_qsize 6, out_qsize 1
2022-11-18 10:15:54,532: INFO: EPOCH 0 - PROGRESS: at 23.92% examples, 578485 words/s, in_qsize 6, out_qsize 1
2022-11-18 10:15:55,549: INFO: EPOCH 0 - PROGRESS: at 24.39% examples, 578653 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:15:56,719: INFO: EPOCH 0 - PROGRESS: at 24.83% examples, 576563 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:15:57,742: INFO: EPOCH 0 - PROGRESS: at 25.15% examples, 575041 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:15:58,761: INFO: EPOCH 0 - PROGRESS: at 25.40% examples, 572737 words/s, in_qsize 8, out_qsize 1
2022-11-18 10:15:59,782: INFO: EPOCH 0 - PROGRESS: at 25.68% examples, 570534 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:16:00,786: INFO: EPOCH 0 - PROGRESS: at 25.98% examples, 568951 words/s, in_qsize 6, out_qsize 1
2

2022-11-18 10:17:07,486: INFO: EPOCH 0 - PROGRESS: at 72.33% examples, 581720 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:08,488: INFO: EPOCH 0 - PROGRESS: at 72.92% examples, 582306 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:09,502: INFO: EPOCH 0 - PROGRESS: at 73.49% examples, 582805 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:10,515: INFO: EPOCH 0 - PROGRESS: at 74.12% examples, 583512 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:11,522: INFO: EPOCH 0 - PROGRESS: at 74.71% examples, 584150 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:12,523: INFO: EPOCH 0 - PROGRESS: at 75.19% examples, 584798 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:13,524: INFO: EPOCH 0 - PROGRESS: at 75.67% examples, 585375 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:17:14,526: INFO: EPOCH 0 - PROGRESS: at 76.25% examples, 585769 words/s, in_qsize 6, out_qsize 1
2022-11-18 10:17:15,547: INFO: EPOCH 0 - PROGRESS: at 76.79% examples, 586316 words/s, in_qsize 7, out_qsize 0
2

2022-11-18 10:18:21,985: INFO: EPOCH 1 - PROGRESS: at 4.22% examples, 653052 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:22,990: INFO: EPOCH 1 - PROGRESS: at 4.54% examples, 654689 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:23,993: INFO: EPOCH 1 - PROGRESS: at 4.89% examples, 656384 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:25,003: INFO: EPOCH 1 - PROGRESS: at 5.28% examples, 657609 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:26,032: INFO: EPOCH 1 - PROGRESS: at 5.63% examples, 658097 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:27,053: INFO: EPOCH 1 - PROGRESS: at 5.99% examples, 655612 words/s, in_qsize 6, out_qsize 1
2022-11-18 10:18:28,056: INFO: EPOCH 1 - PROGRESS: at 6.27% examples, 655236 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:29,058: INFO: EPOCH 1 - PROGRESS: at 6.58% examples, 655071 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:18:30,059: INFO: EPOCH 1 - PROGRESS: at 6.94% examples, 654823 words/s, in_qsize 7, out_qsize 0
2022-11-18

...

2022-11-18 10:25:48,904: INFO: EPOCH 3 - PROGRESS: at 46.82% examples, 627349 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:49,914: INFO: EPOCH 3 - PROGRESS: at 49.74% examples, 627818 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:50,936: INFO: EPOCH 3 - PROGRESS: at 52.84% examples, 628612 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:51,943: INFO: EPOCH 3 - PROGRESS: at 53.20% examples, 627554 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:52,959: INFO: EPOCH 3 - PROGRESS: at 53.65% examples, 627441 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:53,967: INFO: EPOCH 3 - PROGRESS: at 54.10% examples, 627292 words/s, in_qsize 7, out_qsize 1
2022-11-18 10:25:54,974: INFO: EPOCH 3 - PROGRESS: at 54.52% examples, 627446 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:25:55,984: INFO: EPOCH 3 - PROGRESS: at 55.01% examples, 627363 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:25:56,991: INFO: EPOCH 3 - PROGRESS: at 55.81% examples, 627464 words/s, in_qsize 7, out_qsize 0
2

2022-11-18 10:27:03,576: INFO: EPOCH 3 - PROGRESS: at 88.13% examples, 628398 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:04,577: INFO: EPOCH 3 - PROGRESS: at 88.70% examples, 628439 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:27:05,586: INFO: EPOCH 3 - PROGRESS: at 89.23% examples, 628349 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:06,595: INFO: EPOCH 3 - PROGRESS: at 89.86% examples, 628335 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:07,599: INFO: EPOCH 3 - PROGRESS: at 90.43% examples, 628427 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:08,602: INFO: EPOCH 3 - PROGRESS: at 91.07% examples, 628456 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:09,607: INFO: EPOCH 3 - PROGRESS: at 91.49% examples, 627479 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:27:10,615: INFO: EPOCH 3 - PROGRESS: at 92.20% examples, 627467 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:27:11,686: INFO: EPOCH 3 - PROGRESS: at 92.54% examples, 626220 words/s, in_qsize 7, out_qsize 0
2

2022-11-18 10:28:17,776: INFO: EPOCH 4 - PROGRESS: at 9.77% examples, 540061 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:28:18,777: INFO: EPOCH 4 - PROGRESS: at 10.12% examples, 543376 words/s, in_qsize 6, out_qsize 1
2022-11-18 10:28:19,800: INFO: EPOCH 4 - PROGRESS: at 10.59% examples, 546418 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:20,813: INFO: EPOCH 4 - PROGRESS: at 10.99% examples, 549253 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:21,820: INFO: EPOCH 4 - PROGRESS: at 11.35% examples, 551244 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:22,831: INFO: EPOCH 4 - PROGRESS: at 11.74% examples, 551960 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:23,832: INFO: EPOCH 4 - PROGRESS: at 12.19% examples, 554743 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:24,832: INFO: EPOCH 4 - PROGRESS: at 12.59% examples, 556604 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:28:25,838: INFO: EPOCH 4 - PROGRESS: at 12.98% examples, 558105 words/s, in_qsize 7, out_qsize 0
20

2022-11-18 10:29:32,362: INFO: EPOCH 4 - PROGRESS: at 61.29% examples, 610914 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:33,379: INFO: EPOCH 4 - PROGRESS: at 61.64% examples, 610088 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:29:34,387: INFO: EPOCH 4 - PROGRESS: at 62.02% examples, 610000 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:35,405: INFO: EPOCH 4 - PROGRESS: at 62.55% examples, 610445 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:36,407: INFO: EPOCH 4 - PROGRESS: at 63.03% examples, 610955 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:37,413: INFO: EPOCH 4 - PROGRESS: at 63.55% examples, 611390 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:38,415: INFO: EPOCH 4 - PROGRESS: at 64.09% examples, 611976 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:39,437: INFO: EPOCH 4 - PROGRESS: at 64.50% examples, 611735 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:29:40,444: INFO: EPOCH 4 - PROGRESS: at 64.99% examples, 612153 words/s, in_qsize 7, out_qsize 0
2

2022-11-18 10:30:47,166: INFO: EPOCH 4 - PROGRESS: at 97.79% examples, 610480 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:48,174: INFO: EPOCH 4 - PROGRESS: at 98.09% examples, 609485 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:49,179: INFO: EPOCH 4 - PROGRESS: at 98.35% examples, 608228 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:30:50,202: INFO: EPOCH 4 - PROGRESS: at 98.60% examples, 607020 words/s, in_qsize 8, out_qsize 0
2022-11-18 10:30:51,209: INFO: EPOCH 4 - PROGRESS: at 98.84% examples, 605839 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:52,227: INFO: EPOCH 4 - PROGRESS: at 99.02% examples, 604155 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:53,233: INFO: EPOCH 4 - PROGRESS: at 99.27% examples, 603070 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:54,241: INFO: EPOCH 4 - PROGRESS: at 99.55% examples, 602390 words/s, in_qsize 7, out_qsize 0
2022-11-18 10:30:55,258: INFO: EPOCH 4 - PROGRESS: at 99.81% examples, 601222 words/s, in_qsize 7, out_qsize 0
2

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [12]:
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Kemang', 0.6158983707427979), ('Surabaya', 0.6065144538879395), ('Warteg', 0.576435387134552), ('Palmerah', 0.5742366313934326), ('Kemayoran', 0.5598301291465759), ('JAKARTA', 0.5589905977249146), ('Bandung', 0.5589697957038879), ('Petojo', 0.5541684627532959), ('Epicentrum', 0.5540846586227417), ('Ciputat', 0.5464931726455688)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7209160327911377), ('Bogor', 0.698582112789154), ('Tasikmalaya', 0.6477435231208801), ('Jatinangor', 0.638317346572876), ('Garut', 0.6160322427749634), ('Sukabumi', 0.6128425002098083), ('Purwakarta', 0.6105727553367615), ('Cianjur', 0.6073232889175415), ('Baleendah', 0.6068106293678284), ('Dayeuhkolot', 0.589975893497467)]
Kedekatan Yogyakarta-Surakarta: 0.784405529499054
Kedekatan Yogyakarta-Semarang: 0.6524659395217896
makanan-rendang, minuman-?: [('martabak', 0.864336017157527), ('vodka', 0.8601963102629971), ('Brem', 0.8566517426300826), ('Minuman', 0.8549460409933962), ('penganan